# 3-神经网络

神经网络的典型训练过程如下:

* 定义神经网络模型,它有一些可学习的参数(或者权重);
* 在数据集上迭代;
* 通过神经网络处理输入;
* 计算损失(输出结果和正确值的差距大小)
* 将梯度反向传播会网络的参数;
* 更新网络的参数,主要使用如下简单的更新原则:`weight = weight - learning_rate * gradient`

In [1]:
# 定义网络
import torch
import torch.nn as nn
import torch.nn.functional as F


In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [7]:
net = Net()
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [8]:
params = list(net.parameters())
len(params)

10

In [9]:
params[0].size()

torch.Size([6, 1, 5, 5])

In [14]:
input = torch.randn(1,1,32,32)
out = net(input)
out

tensor([[ 0.0471,  0.0344, -0.0182,  0.0431, -0.1320,  0.0526, -0.1145,  0.0472,
          0.0586, -0.0109]], grad_fn=<AddmmBackward>)

In [15]:
# 将所有参数的梯度缓存清零,然后进行随机梯度的的反向传播.
net.zero_grad()
out.backward(torch.randn(1,10))

* 注意

* `torch.nn` 只支持小批量输入,整个`torch.nn`包都只支持小批量样本,而不支持单个样本
* 例如,`nn.Conv2d`将接受一个4维的张量,每一维分别是$nSamples\times nChannels\times Height\times Width$(样本数\*通道数\*高\*宽).
* 如果你有单个样本,只需使用`input.unsqueeze(0)`来添加其它的维数.

* `torch.Tensor`-支持自动编程操作（如`backward()`）的多维数组。 同时保持梯度的张量。
* `nn.Module`-神经网络模块.封装参数,移动到GPU上运行,导出,加载等
* `nn.Parameter`-一种张量,当把它赋值给一个`Module`时,被自动的注册为参数.

* `autograd.Function`-实现一个自动求导操作的前向和反向定义, 每个张量操作都会创建至少一个`Function`节点，该节点连接到创建张量并对其历史进行编码的函数

### 损失函数
一个损失函数接受一对(output, target)作为输入(output为网络的输出,target为实际值),计算一个值来估计网络的输出和目标值相差多少。

在nn包中有几种不同的损失函数.一个简单的损失函数是:`nn.MSELoss`,它计算输入和目标之间的均方误差。


In [16]:
output = net(input)
target = torch.randn(10)
target = target.view(1,-1)
criterion = nn.MSELoss()
loss = criterion(output,target)
loss

tensor(0.9104, grad_fn=<MseLossBackward>)

input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
     -> view -> linear -> relu -> linear -> relu -> linear
     -> MSELoss
     -> loss
所以, 当你调用`loss.backward()`,整个图被区分为损失以及图中所有具有`requires_grad = True`的张量，并且其`.grad` 张量的梯度累积。

In [18]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

### 反向传播
为了反向传播误差,我们所需做的是调用`loss.backward()`.你需要清除已存在的梯度,否则梯度将被累加到已存在的梯度。

现在,我们将调用`loss.backward()`,并查看conv1层的偏置项在反向传播前后的梯度。

In [19]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0020,  0.0030, -0.0059,  0.0200,  0.0113, -0.0031])


### 更新权重
实践中最简单的更新规则是随机梯度下降(SGD)．

`weight=weight−learning_rate∗gradient`


In [20]:
# 接下来使用python代码实现

In [21]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

然而,当你使用神经网络是,你想要使用各种不同的更新规则,比如`SGD,Nesterov-SGD`,`Adam`, `RMSPROP`等.为了能做到这一点,我们构建了一个包`torch.optim`实现了所有的这些规则.使用他们非常简单:

In [22]:
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [23]:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

使用`optimizer.zero_grad()`手动将梯度缓冲区设置为零。 这是因为梯度是反向传播部分中的说明那样是累积的。